# My Utils

In [ ]:
import bayes3d as b3d
import bayes3d.genjax
import genjax
from genjax import gen
import jax.numpy as jnp
import jax
import os
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import jax.tree_util as jtu
import numpy as np
from genjax._src.core.transforms.incremental import UnknownChange
from genjax._src.core.transforms.incremental import NoChange
from genjax._src.core.transforms.incremental import Diff
import trimesh
import pythreejs as p3s

In [ ]:
def rot2d(hd): return jnp.array([
    [jnp.cos(hd), -jnp.sin(hd)], 
    [jnp.sin(hd),  jnp.cos(hd)]
    ])

def pack_2dpose(x,hd): return jnp.concatenate([x,jnp.array([hd])])

def apply_2dpose(p, ys): 
    return ys@rot2d(p[2] - jnp.pi/2).T + p[:2]

def unit_vec(hd): return jnp.array([jnp.cos(hd), jnp.sin(hd)])

def adjust_angle(hd): return (hd + jnp.pi)%(2*jnp.pi) - jnp.pi

def keysplit(key, *ns):
    if len(ns) == 0:

        return jax.random.split(key, 1)[0]
    
    else:

        keys = []
        for n in ns:
            if n == 1: keys.append(jax.random.split(key, 1)[0])
            else: keys.append(jax.random.split(key, n))

        return keys


logsumexp = jax.scipy.special.logsumexp


In [ ]:
from builtins import property as _property, tuple as _tuple
from typing import Any

class Args(tuple):
    def __new__(cls, *args, **kwargs):
        return _tuple.__new__(cls, list(args) + list(kwargs.values()))
    
    def __init__(self, *args, **kwargs):
        self._d = dict()
        for k,v in kwargs.items():
            self._d[k] = v
            setattr(self, k, v)

    def __getitem__(self, k: str) -> Any:
        return self._d[k]

In [ ]:
def argdiffs(args, other=None):
    if other is None: 
        return tuple(map(lambda v: Diff(v, UnknownChange), args))
    else:
        return tuple(map(lambda v: Diff(v, UnknownChange), args))

In [ ]:
def argmax_axes(a, axes=None):
    if axes is None: return jnp.argmax(a)
    
    n = len(axes)        
    axes_  = set(range(a.ndim))
    axes_0 = axes
    axes_1 = sorted(axes_ - set(axes_0))    
    axes_  = axes_0 + axes_1

    b = jnp.transpose(a, axes=axes_)
    c = b.reshape(np.prod(b.shape[:n]), -1)

    I = jnp.argmax(c, axis=0)
    I = jnp.array([jnp.unravel_index(i, b.shape[:n]) for i in I]).reshape(b.shape[n:] + (n,))

    return  I


test_shape = (3, 99, 5, 9)
a = jnp.arange(np.prod(test_shape)).reshape(test_shape)

I = argmax_axes(a, axes=[0,1])
I.shape